# Ejercicios Prácticos: Optimización de Hiperparámetros

Este notebook contiene ejercicios para practicar el uso de las principales herramientas de optimización de hiperparámetros en Deep Learning.

**Instrucciones generales:**
- Lee cuidadosamente cada enunciado antes de comenzar
- Completa el código en las celdas indicadas
- Ejecuta las celdas de verificación cuando estén disponibles
- Los ejercicios están ordenados por dificultad creciente

**Datasets utilizados:**
- MNIST (dígitos manuscritos)
- Fashion MNIST (prendas de ropa)
- CIFAR-10 (imágenes a color)

In [ ]:
# Instalación de dependencias
!pip install keras-tuner optuna "ray[tune]" plotly -q

In [ ]:
# Importaciones comunes
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Silenciar warnings
tf.get_logger().setLevel('ERROR')

print(f"TensorFlow version: {tf.__version__}")

---
## Parte 1: Keras Tuner

### Ejercicio 1.1: Búsqueda Básica de Hiperparámetros

**Objetivo:** Familiarizarse con la sintaxis básica de Keras Tuner.

**Enunciado:**

Tienes un modelo simple para clasificar el dataset Fashion MNIST. Tu tarea es modificar la función `build_model` para que Keras Tuner pueda buscar automáticamente:

1. El número de neuronas en la capa oculta (entre 64 y 256, en pasos de 64)
2. La tasa de dropout (entre 0.2 y 0.5, en pasos de 0.1)
3. La tasa de aprendizaje (entre 0.001 y 0.01, usando escala logarítmica)

**Pistas:**
- Usa `hp.Int()` para valores enteros
- Usa `hp.Float()` para valores decimales
- El parámetro `sampling='log'` permite búsqueda en escala logarítmica

In [ ]:
# Cargar datos
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(f"Clases: {np.unique(y_train)}")
print(f"Shape: {x_train.shape}")

In [ ]:
import keras_tuner as kt

def build_model(hp):
    """
    TODO: Completa esta función para que los hiperparámetros sean buscables.
    
    Hiperparámetros a buscar:
    - units: número de neuronas (64 a 256, paso 64)
    - dropout: tasa de dropout (0.2 a 0.5, paso 0.1)
    - learning_rate: tasa de aprendizaje (0.001 a 0.01, escala log)
    """
    model = keras.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        # TODO: Modificar la siguiente línea para buscar el número de unidades
        layers.Dense(128, activation='relu'),
        # TODO: Modificar la siguiente línea para buscar el dropout
        layers.Dropout(0.3),
        layers.Dense(10, activation='softmax')
    ])
    
    # TODO: Modificar para buscar el learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Tu código aquí


In [ ]:
# Configura y ejecuta el tuner con RandomSearch
# TODO: Crear un tuner con kt.RandomSearch
# - objective: 'val_accuracy'
# - max_trials: 10
# - directory: 'ejercicio_1_1'
# - project_name: 'fashion_mnist'

# Tu código aquí


In [ ]:
# Ejecuta la búsqueda
# TODO: Llamar a tuner.search() con:
# - epochs=5
# - validation_split=0.2

# Tu código aquí


In [ ]:
# Muestra los mejores hiperparámetros
# TODO: Obtener e imprimir los mejores hiperparámetros

# Tu código aquí


---
### Ejercicio 1.2: Búsqueda de Arquitectura Variable

**Objetivo:** Aprender a buscar arquitecturas con número variable de capas.

**Enunciado:**

Crea un modelo donde Keras Tuner pueda decidir:

1. El número de capas ocultas (entre 1 y 4)
2. Para cada capa:
   - Número de neuronas (32, 64, 128, o 256)
   - Función de activación ('relu' o 'tanh')
3. Si usar o no BatchNormalization después de cada capa densa

**Pistas:**
- Usa `hp.Choice()` para seleccionar de una lista de opciones
- Usa `hp.Boolean()` para decisiones verdadero/falso
- Usa un bucle `for` con `range(hp.Int(...))` para capas variables

In [ ]:
def build_variable_model(hp):
    """
    TODO: Implementa un modelo con arquitectura variable.
    
    Hiperparámetros:
    - num_layers: 1 a 4 capas
    - units_i: neuronas por capa (32, 64, 128, 256)
    - activation_i: función de activación ('relu', 'tanh')
    - use_batchnorm: usar BatchNormalization (True/False)
    """
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    # TODO: Implementar la búsqueda de arquitectura variable
    # Tu código aquí
    
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Tu código aquí


In [ ]:
# Configura un tuner con BayesianOptimization y ejecuta la búsqueda
# TODO: Crear tuner, ejecutar búsqueda y mostrar resultados

# Tu código aquí


---
### Ejercicio 1.3: Hyperband para Búsqueda Eficiente

**Objetivo:** Utilizar el algoritmo Hyperband para búsquedas más eficientes.

**Enunciado:**

El algoritmo Hyperband es más eficiente que la búsqueda aleatoria porque asigna más recursos a las configuraciones prometedoras. Tu tarea es:

1. Crear un modelo con los siguientes hiperparámetros buscables:
   - Número de filtros en capas Conv2D (16, 32, 64)
   - Tamaño del kernel (3 o 5)
   - Usar o no MaxPooling después de cada Conv2D
   - Número de neuronas en la capa densa (64 a 256)
   - Learning rate (1e-4 a 1e-2)

2. Configurar un tuner Hyperband con:
   - `max_epochs=30`
   - `factor=3` (factor de reducción)
   - `hyperband_iterations=2`

**Dataset:** MNIST reshapeado a (28, 28, 1) para usar Conv2D

In [ ]:
# Preparar datos para CNN
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

print(f"Shape de entrenamiento: {x_train.shape}")

In [ ]:
def build_cnn_model(hp):
    """
    TODO: Implementa una CNN con hiperparámetros buscables.
    
    Hiperparámetros:
    - filters: número de filtros (16, 32, 64)
    - kernel_size: tamaño del kernel (3, 5)
    - use_pooling: usar MaxPooling (True/False)
    - dense_units: neuronas en capa densa (64 a 256, paso 32)
    - learning_rate: tasa de aprendizaje (1e-4 a 1e-2, log)
    """
    model = keras.Sequential()
    
    # TODO: Implementar la CNN con hiperparámetros buscables
    # Tu código aquí
    
    return model

# Tu código aquí


In [ ]:
# Configura y ejecuta el tuner Hyperband
# TODO: Crear kt.Hyperband tuner y ejecutar búsqueda

# Tu código aquí


---
## Parte 2: Optuna

### Ejercicio 2.1: Estudio Básico con Optuna

**Objetivo:** Aprender la estructura básica de Optuna: Study, Trial y Objective.

**Enunciado:**

Implementa una función objetivo para Optuna que entrene un modelo MLP para Fashion MNIST. La función debe:

1. Sugerir los siguientes hiperparámetros:
   - `n_layers`: número de capas (1 a 3)
   - `n_units`: neuronas por capa (32 a 256)
   - `dropout`: tasa de dropout (0.1 a 0.5)
   - `learning_rate`: tasa de aprendizaje (1e-5 a 1e-2, escala log)
   - `batch_size`: tamaño de batch (32, 64, o 128)

2. Entrenar el modelo por 10 épocas

3. Retornar la accuracy de validación

**Pistas:**
- `trial.suggest_int()` para enteros
- `trial.suggest_float(..., log=True)` para escala logarítmica
- `trial.suggest_categorical()` para elegir de una lista

In [ ]:
import optuna

# Cargar datos
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Separar validación
x_val, y_val = x_train[:10000], y_train[:10000]
x_train, y_train = x_train[10000:], y_train[10000:]

In [ ]:
def objective(trial):
    """
    TODO: Implementa la función objetivo para Optuna.
    
    Debe:
    1. Sugerir hiperparámetros usando trial.suggest_*
    2. Construir y entrenar el modelo
    3. Retornar la accuracy de validación
    """
    # TODO: Sugerir hiperparámetros
    # Tu código aquí
    
    # TODO: Construir modelo
    # Tu código aquí
    
    # TODO: Entrenar y retornar accuracy
    # Tu código aquí
    
    pass  # Eliminar esta línea cuando implementes la función

# Tu código aquí


In [ ]:
# Crear y ejecutar el estudio
# TODO: Crear estudio con optuna.create_study(direction='maximize')
# TODO: Ejecutar study.optimize() con n_trials=15

# Tu código aquí


In [ ]:
# Mostrar resultados y mejores hiperparámetros
# TODO: Imprimir study.best_trial.value y study.best_trial.params

# Tu código aquí


---
### Ejercicio 2.2: Pruning con Optuna

**Objetivo:** Implementar poda (pruning) para detener entrenamientos no prometedores.

**Enunciado:**

Modifica la función objetivo anterior para implementar pruning:

1. Reportar la accuracy de validación después de cada época usando `trial.report()`
2. Verificar si el trial debe ser podado usando `trial.should_prune()`
3. Si debe ser podado, lanzar `optuna.TrialPruned()`

Configura el estudio con un `MedianPruner` que:
- Espere 3 trials antes de empezar a podar (`n_startup_trials=3`)
- Espere 3 épocas antes de podar un trial (`n_warmup_steps=3`)

**Pista:** Necesitarás crear un callback personalizado de Keras para reportar métricas a Optuna.

In [ ]:
def objective_with_pruning(trial):
    """
    TODO: Implementa la función objetivo con pruning.
    
    Debe incluir:
    1. Sugerencia de hiperparámetros
    2. Un callback que reporte métricas y verifique pruning
    3. Manejo de la excepción TrialPruned
    """
    # TODO: Implementar con pruning
    # Tu código aquí
    
    pass  # Eliminar esta línea cuando implementes la función

# Tu código aquí


In [ ]:
# Crear estudio con pruner y ejecutar
# TODO: Crear estudio con MedianPruner
# TODO: Ejecutar optimización

# Tu código aquí


In [ ]:
# Analizar cuántos trials fueron podados
# TODO: Contar trials completados vs podados

# Tu código aquí


---
### Ejercicio 2.3: Visualizaciones de Optuna

**Objetivo:** Utilizar las herramientas de visualización de Optuna para analizar resultados.

**Enunciado:**

Usando el estudio del ejercicio anterior (o creando uno nuevo con al menos 20 trials), genera las siguientes visualizaciones:

1. **Historia de optimización**: Muestra cómo evoluciona el mejor valor encontrado
2. **Importancia de parámetros**: Identifica qué hiperparámetros tienen mayor impacto
3. **Coordenadas paralelas**: Visualiza las relaciones entre hiperparámetros
4. **Gráfico de contorno**: Para dos hiperparámetros seleccionados

Después de generar las visualizaciones, responde:
- ¿Cuál es el hiperparámetro más importante?
- ¿Hay alguna correlación visible entre hiperparámetros?

In [ ]:
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_contour
)

# TODO: Generar las 4 visualizaciones
# Tu código aquí


**Respuestas:**

1. ¿Cuál es el hiperparámetro más importante?
   - *Tu respuesta aquí*

2. ¿Hay alguna correlación visible entre hiperparámetros?
   - *Tu respuesta aquí*

---
## Parte 3: Ray Tune

### Ejercicio 3.1: Configuración Básica de Ray Tune

**Objetivo:** Aprender a configurar espacios de búsqueda y ejecutar experimentos con Ray Tune.

**Enunciado:**

Configura un experimento de Ray Tune para optimizar un modelo de clasificación en CIFAR-10. Debes:

1. Definir un espacio de búsqueda con:
   - `conv_filters`: [32, 64, 128] (categorical)
   - `dense_units`: 64 a 512 (uniforme)
   - `learning_rate`: 1e-4 a 1e-1 (log-uniforme)
   - `batch_size`: [32, 64, 128] (categorical)

2. Crear una función de entrenamiento que:
   - Construya una CNN simple (2-3 capas conv + dense)
   - Reporte métricas a Ray Tune usando `tune.report()`

3. Ejecutar la búsqueda con 10 trials

**Nota:** CIFAR-10 tiene imágenes de 32x32x3

In [ ]:
import ray
from ray import tune

# Inicializar Ray
ray.init(ignore_reinit_error=True)

In [ ]:
# Cargar CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

print(f"Shape: {x_train.shape}")
print(f"Clases: {np.unique(y_train)}")

In [ ]:
# TODO: Definir el espacio de búsqueda
search_space = {
    # Tu código aquí
}

# Tu código aquí


In [ ]:
def train_cifar(config):
    """
    TODO: Implementa la función de entrenamiento para Ray Tune.
    
    Debe:
    1. Cargar los datos de CIFAR-10
    2. Construir una CNN usando los hiperparámetros de config
    3. Entrenar y reportar métricas con tune.report()
    """
    # TODO: Implementar
    # Tu código aquí
    
    pass  # Eliminar esta línea cuando implementes la función

# Tu código aquí


In [ ]:
# Ejecutar la búsqueda
# TODO: Usar tune.run() con la configuración apropiada

# Tu código aquí


In [ ]:
# Mostrar mejores resultados
# TODO: Obtener y mostrar la mejor configuración

# Tu código aquí


---
### Ejercicio 3.2: Scheduler ASHA

**Objetivo:** Utilizar el scheduler ASHA para detención temprana eficiente.

**Enunciado:**

Modifica el ejercicio anterior para usar el scheduler ASHA (Asynchronous Successive Halving Algorithm):

1. Configura un `ASHAScheduler` con:
   - `max_t=50` (máximo de épocas)
   - `grace_period=10` (épocas mínimas antes de podar)
   - `reduction_factor=3`

2. Aumenta el número de trials a 20

3. Compara el tiempo total de búsqueda con el ejercicio anterior

**Pregunta:** ¿Cuántos trials fueron detenidos tempranamente? ¿Cuál fue el ahorro de tiempo aproximado?

In [ ]:
from ray.tune.schedulers import ASHAScheduler

# TODO: Configurar ASHAScheduler
# Tu código aquí


In [ ]:
# TODO: Ejecutar búsqueda con ASHA
# Tu código aquí


In [ ]:
# TODO: Analizar resultados y trials detenidos
# Tu código aquí


**Respuestas:**

1. ¿Cuántos trials fueron detenidos tempranamente?
   - *Tu respuesta aquí*

2. ¿Cuál fue el ahorro de tiempo aproximado?
   - *Tu respuesta aquí*

In [ ]:
# Limpiar Ray
ray.shutdown()

---
## Parte 4: TensorBoard HParams

### Ejercicio 4.1: Registro de Experimentos

**Objetivo:** Aprender a registrar hiperparámetros y métricas en TensorBoard.

**Enunciado:**

Crea un experimento sistemático que pruebe las siguientes combinaciones de hiperparámetros:

1. **Optimizadores**: Adam, SGD con momentum, RMSprop
2. **Learning rates**: 0.001, 0.01, 0.1
3. **Tamaños de batch**: 32, 128

Para cada combinación:
- Entrena un modelo MLP simple en MNIST por 5 épocas
- Registra los hiperparámetros y la accuracy final en TensorBoard

**Total de experimentos:** 3 × 3 × 2 = 18

In [ ]:
from tensorboard.plugins.hparams import api as hp
import datetime
import os
import shutil

# Limpiar logs anteriores
if os.path.exists("logs/ejercicio_4"):
    shutil.rmtree("logs/ejercicio_4")

In [ ]:
# TODO: Definir los hiperparámetros con hp.HParam
# HP_OPTIMIZER = hp.HParam(...)
# HP_LEARNING_RATE = hp.HParam(...)
# HP_BATCH_SIZE = hp.HParam(...)

# Tu código aquí


In [ ]:
# TODO: Configurar el directorio de logs y hp.hparams_config

# Tu código aquí


In [ ]:
# Cargar datos
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
def run_experiment(hparams, run_dir):
    """
    TODO: Implementa la función que ejecuta un experimento.
    
    Debe:
    1. Crear el modelo
    2. Configurar el optimizador según hparams
    3. Entrenar por 5 épocas
    4. Retornar la accuracy en test
    """
    # Tu código aquí
    
    pass  # Eliminar esta línea cuando implementes la función

# Tu código aquí


In [ ]:
# TODO: Ejecutar todos los experimentos en un bucle
# Registrar hiperparámetros y métricas con hp.hparams() y tf.summary.scalar()

# Tu código aquí


In [ ]:
# Cargar TensorBoard
%load_ext tensorboard

In [ ]:
# Visualizar en TensorBoard
%tensorboard --logdir logs/ejercicio_4

---
### Ejercicio 4.2: Análisis de Resultados

**Objetivo:** Analizar los resultados usando la interfaz de TensorBoard HParams.

**Enunciado:**

Usando la visualización de TensorBoard del ejercicio anterior, responde las siguientes preguntas:

1. ¿Cuál combinación de hiperparámetros dio la mejor accuracy?
2. ¿Qué optimizador tuvo mejor rendimiento promedio?
3. ¿El learning rate de 0.1 funcionó bien con todos los optimizadores?
4. ¿Hubo diferencia significativa entre batch size 32 y 128?

**Instrucciones:**
- En TensorBoard, ve a la pestaña "HPARAMS"
- Usa la tabla para ordenar por accuracy
- Usa el gráfico de coordenadas paralelas para ver patrones
- Filtra por diferentes valores para responder las preguntas

**Respuestas:**

1. ¿Cuál combinación de hiperparámetros dio la mejor accuracy?
   - *Tu respuesta aquí*

2. ¿Qué optimizador tuvo mejor rendimiento promedio?
   - *Tu respuesta aquí*

3. ¿El learning rate de 0.1 funcionó bien con todos los optimizadores?
   - *Tu respuesta aquí*

4. ¿Hubo diferencia significativa entre batch size 32 y 128?
   - *Tu respuesta aquí*

---
## Parte 5: Ejercicios Integradores

### Ejercicio 5.1: Comparación de Herramientas

**Objetivo:** Comparar las diferentes herramientas en un mismo problema.

**Enunciado:**

Realiza una búsqueda de hiperparámetros para el mismo modelo y dataset usando tres herramientas diferentes:

1. **Keras Tuner** con BayesianOptimization
2. **Optuna** con MedianPruner
3. **Ray Tune** con ASHAScheduler

**Configuración común:**
- Dataset: Fashion MNIST
- Modelo: MLP con 1-3 capas ocultas
- Hiperparámetros: número de capas, neuronas por capa, dropout, learning rate
- Número de trials: 15 por herramienta
- Épocas máximas: 20

**Métricas a comparar:**
- Mejor accuracy encontrada
- Tiempo total de búsqueda
- Facilidad de uso (subjetivo)

In [ ]:
# Datos comunes para todos los experimentos
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_val, y_val = x_train[:10000], y_train[:10000]
x_train, y_train = x_train[10000:], y_train[10000:]

In [ ]:
import time

results = {
    'keras_tuner': {'best_accuracy': None, 'time': None},
    'optuna': {'best_accuracy': None, 'time': None},
    'ray_tune': {'best_accuracy': None, 'time': None}
}

In [ ]:
# TODO: Implementar búsqueda con Keras Tuner
print("=" * 50)
print("KERAS TUNER")
print("=" * 50)

start_time = time.time()

# Tu código aquí

results['keras_tuner']['time'] = time.time() - start_time
# results['keras_tuner']['best_accuracy'] = ...

In [ ]:
# TODO: Implementar búsqueda con Optuna
print("=" * 50)
print("OPTUNA")
print("=" * 50)

start_time = time.time()

# Tu código aquí

results['optuna']['time'] = time.time() - start_time
# results['optuna']['best_accuracy'] = ...

In [ ]:
# TODO: Implementar búsqueda con Ray Tune
print("=" * 50)
print("RAY TUNE")
print("=" * 50)

ray.init(ignore_reinit_error=True)
start_time = time.time()

# Tu código aquí

results['ray_tune']['time'] = time.time() - start_time
# results['ray_tune']['best_accuracy'] = ...

ray.shutdown()

In [ ]:
# Mostrar comparación
print("\n" + "=" * 50)
print("COMPARACIÓN DE RESULTADOS")
print("=" * 50)

for tool, data in results.items():
    print(f"\n{tool.upper()}:")
    print(f"  Mejor accuracy: {data['best_accuracy']:.4f}")
    print(f"  Tiempo total: {data['time']:.2f} segundos")

---
### Ejercicio 5.2: Optimización de CNN para CIFAR-10

**Objetivo:** Aplicar todo lo aprendido en un problema más complejo.

**Enunciado:**

Diseña y ejecuta una búsqueda completa de hiperparámetros para una CNN en CIFAR-10 usando la herramienta de tu preferencia. El objetivo es alcanzar al menos **70% de accuracy** en el conjunto de test.

**Requisitos:**

1. La arquitectura debe incluir:
   - Al menos 2 bloques convolucionales
   - Capas de pooling
   - Al menos una capa densa
   - Regularización (dropout y/o batch normalization)

2. Hiperparámetros a buscar (mínimo 5):
   - Número de filtros
   - Tamaño de kernel
   - Tipo de pooling (Max vs Average)
   - Dropout rate
   - Learning rate
   - Optimizador
   - Número de capas densas
   - Uso de batch normalization

3. Utiliza detención temprana o pruning para eficiencia

4. Documenta tu proceso y justifica tus decisiones

**Entregables:**
- Código completo de la búsqueda
- Mejores hiperparámetros encontrados
- Accuracy final en test
- Análisis de qué hiperparámetros fueron más importantes

In [ ]:
# Cargar CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

# Separar validación
x_val, y_val = x_train[:5000], y_train[:5000]
x_train, y_train = x_train[5000:], y_train[5000:]

print(f"Entrenamiento: {x_train.shape}")
print(f"Validación: {x_val.shape}")
print(f"Test: {x_test.shape}")

In [ ]:
# TODO: Implementar tu solución aquí
# Tu código aquí


In [ ]:
# TODO: Entrenar modelo final con mejores hiperparámetros
# Tu código aquí


In [ ]:
# TODO: Evaluar en test y mostrar resultados
# Tu código aquí


**Análisis y conclusiones:**

1. ¿Qué herramienta elegiste y por qué?
   - *Tu respuesta aquí*

2. ¿Cuáles fueron los mejores hiperparámetros encontrados?
   - *Tu respuesta aquí*

3. ¿Qué hiperparámetros tuvieron mayor impacto en el rendimiento?
   - *Tu respuesta aquí*

4. ¿Alcanzaste el objetivo del 70%? Si no, ¿qué más intentarías?
   - *Tu respuesta aquí*

5. ¿Cuánto tiempo tomó la búsqueda y cuántos trials se ejecutaron?
   - *Tu respuesta aquí*

---
## Ejercicio Bonus: Transfer Learning con Optimización

**Objetivo:** Combinar transfer learning con búsqueda de hiperparámetros.

**Enunciado:**

Utiliza un modelo preentrenado (MobileNetV2) como base y optimiza los hiperparámetros de las capas que agregues encima. 

**Hiperparámetros a buscar:**
- Número de capas densas adicionales (1-3)
- Neuronas en cada capa
- Si descongelar las últimas N capas del modelo base
- Learning rate para fine-tuning

**Dataset:** CIFAR-10 redimensionado a 96x96 para MobileNetV2

**Nota:** Este ejercicio requiere más recursos computacionales. Considera usar Google Colab con GPU si tu máquina local es lenta.

In [ ]:
# TODO: Implementar transfer learning con búsqueda de hiperparámetros
# Este es un ejercicio avanzado opcional

# Tu código aquí


---
## Resumen y Reflexión Final

Después de completar estos ejercicios, reflexiona sobre las siguientes preguntas:

1. ¿Cuál herramienta te resultó más intuitiva de usar?

2. ¿En qué situaciones usarías cada herramienta?

3. ¿Qué aprendiste sobre la importancia relativa de los diferentes hiperparámetros?

4. ¿Cómo cambiaría tu enfoque si tuvieras acceso a múltiples GPUs?

5. ¿Qué estrategias usarías para reducir el tiempo de búsqueda en proyectos reales?